In [14]:
import numpy as np
import pandas as pd
from utils import *
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from copy import copy
import torch.optim as optim
from tqdm import tqdm
df = pd.read_csv('C:\\Users\\LENOVO\\Downloads\\metadata.csv', index_col=0)



In [15]:
df

,ID,lab_mean,lab_std,lab_mode,hsv_mean,hsv_std,hsv_mode,black_per,hour,date,Y
0,11_junio_1_10_A,5.579561,2.392275,6.113646,0.289046,0.080507,0.359928,0.271260,13,11_junio,N_Control
1,11_junio_1_10_B,6.014204,2.194136,6.112457,0.285616,0.089140,0.359079,0.240148,13,11_junio,P_Control
2,11_junio_1_10_C,5.565204,2.388955,6.112457,0.279931,0.070453,0.300556,0.249660,13,11_junio,K_Control
3,11_junio_1_10_D,5.603238,2.373519,6.112457,0.288891,0.058296,0.281048,0.220348,13,11_junio,Control
4,11_junio_1_10_E,5.942148,2.292411,6.112457,0.267263,0.090707,0.360951,0.206660,13,11_junio,H50%
...,...,...,...,...,...,...,...,...,...,...,...
2641,29_marzo_2_9_C,5.501778,2.294248,5.615772,0.219387,0.074460,0.186111,0.239337,13,29_marzo,K_Control
2642,29_marzo_2_9_D,4.585215,2.236201,2.528976,0.255162,0.060826,0.272256,0.257425,13,29_marzo,Control
2643,29_marzo_2_9_E,5.246325,2.243122,4.819172,0.265598,0.085237,0.265514,0.224837,13,29_marzo,H50%
2644,29_marzo_2_9_F,4.818452,2.308415,2.528948,0.264396,0.084294,0.272465,0.245659,13,29_marzo,H75%


In [16]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

##SELECCION DE N Y CLASES
N_data = df[(df['Y']=='N_Control') | (df['Y']=='N_Exceso') | (df['Y']=='N_Deficiencia')]

In [17]:
N_data = N_data.iloc[:,[1,2,3,4,5,6,7,10]]
n2clas={'N_Deficiencia':0, 'N_Control':1, 'N_Exceso':2}
N_data.Y.replace(n2clas, inplace=True)
N_data

,lab_mean,lab_std,lab_mode,hsv_mean,hsv_std,hsv_mode,black_per,Y
0,5.579561,2.392275,6.113646,0.289046,0.080507,0.359928,0.271260,1
6,5.438281,2.349311,6.114479,0.301601,0.065519,0.301523,0.294979,1
19,5.768403,2.359222,6.112457,0.272714,0.080226,0.303015,0.258090,1
26,5.352736,2.477107,6.113646,0.268448,0.077573,0.279778,0.297926,1
33,6.292614,2.239615,6.112457,0.274107,0.083767,0.308799,0.198454,0
...,...,...,...,...,...,...,...,...
2611,4.716149,2.384748,2.827673,0.265580,0.080164,0.272718,0.217523,2
2618,4.667308,2.329164,2.827673,0.268359,0.086640,0.272465,0.209316,2
2625,4.795935,2.339041,3.425122,0.263638,0.089320,0.271694,0.240542,2
2632,5.109454,2.309430,4.918747,0.235589,0.073833,0.253598,0.221984,1


In [88]:
def train(idx, ep = 100):
    idx_result = {'idx_used':copy(idx), 'f1': 0, 'acc':0}
    idx_copy = copy(idx)
    aux_f1_idx = []
    aux_ac_idx = []
    n_f = len(idx_copy)
    idx_copy.extend([-1])
    a = datapaltas(df = N_data.iloc[:, idx_copy], y_idx=-1)
    EPOCHS = ep
    
    skf = StratifiedKFold()
    ## y_idx -2
    skf.get_n_splits(N_data, N_data.iloc[:,-1])
    for fold, (train_ids, test_ids) in enumerate(skf.split(N_data, N_data.iloc[:,-1])):
        l_v = []
        a_v = []
        f_v = []
        #print(f'Fold {fold+1}')
        #print('------------------------')
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_loader = torch.utils.data.DataLoader(
                          a, 
                          batch_size=64, sampler=train_subsampler)
        test_loader = torch.utils.data.DataLoader(
                          a,
                          batch_size=1, sampler=test_subsampler)
        model = SimpleClassification(n_features=n_f)
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 0.001)
        itt = tqdm(range(EPOCHS))
        for i in itt:
            loss_epoch = 0
            acc_train, acc_test = 0.0, 0.0
            f1_train, f1_test = 0.0, 0.0
            model.train()
            for X, y in train_loader:
                X, y = X.to(device), y.to(device).flatten()
                optimizer.zero_grad()
                y_pred = model(X)
                loss = criterion(y_pred, y)
                acc = multi_acc(y_pred, y)
                f1 = F1_score(y_pred, y)
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
                acc_train += acc.item()
                f1_train += f1.item()

            with torch.no_grad():

                for X, y in test_loader:
                    X, y = X.to(device), y.to(device).flatten()
                    model.eval()
                    output = model(X)
                    acc = multi_acc(output, y)
                    f1 = F1_score(output, y)
                    acc_test += acc.item()
                    f1_test += f1.item()

            itt.set_description("Acc train: %.2f Acc test: %.2f F1 train: %.2f F1 test: %.2f" % (acc_train/len(train_loader), acc_test/len(test_loader), f1_train/len(train_loader), f1_test/len(test_loader)))

            l_v.append(loss_epoch)
            a_v.append(acc_test/len(test_loader))
            f_v.append(f1_test/len(test_loader))
            
            #print(f'Loss {i+1}: {loss_epoch}')
            #print(f'Acc {i+1}: {acc_epoch/len(train_loader)}')
            #print(f'F1 {i+1}: {f1_epoch/len(train_loader)}')
        #print(a_v, f_v)
        aux_f1_idx.append(f_v)
        aux_ac_idx.append(a_v)
        
    aux_ac_idx = np.array(aux_ac_idx)
    aux_f1_idx = np.array(aux_f1_idx)
    #print(aux_ac_idx, aux_f1_idx)
    idx_result['f1'] = max(np.mean(aux_f1_idx, axis = 0))
    idx_result['acc'] = max(np.mean(aux_ac_idx, axis = 0))
    return idx_result
        #plt.figure(), plt.plot(l_v), plt.title('Loss by epoch')
        #plt.figure(), plt.plot(a_v), plt.title('Accuracy test by epoch')
        #plt.figure(), plt.plot(f_v), plt.title('F1 test by epoch')
        #plt.show()
    
N_FEATURES = 7

def yolo(initial_feat, i, selector, total_log, ev):
    print('a')
    initial_feat.append(i)
    print(f"=======================================Using {initial_feat} ===============")
    res = train(initial_feat[1:], ep = 100)
    initial_feat.pop()
    print(selector)
    selector.update({i:res[ev]})
    #print(selector)
    total_log.append(res)
    #return 0
    return selector
v_yolo=np.vectorize(pyfunc=yolo,signature="(i),(j),(),(),()->()",otypes=[object])

###SELECTOR
def select_k_best(k = 10, ev = 'f1'):
    """
    ev = 'acc' or 'f1'
    """
    total_log = []
    list_features = list(range(N_FEATURES))
    initial_feat = [-200]
    #initial_feat = np.arange(len(list_features))
   # initial_feat =np.empty((1,))
    while(len(initial_feat)<k):
        selector = {}
        #for i in list_features:
        #selector = yolo(initial_feat, 0, selector, total_log, ev)
        s=v_yolo(np.tile(np.array(initial_feat), (len(list_features),1)), np.array(list_features).reshape(-1,1), selector, total_log, ev)
        print(s)
        print('iter')
        print(selector)                                        
        best_feature = max(selector, key=selector.get)
        initial_feat.append(best_feature)
        list_features.remove(best_feature)
        print("================================================")
        print(f"=================ADEDD {max(selector, key=selector.get)} TO BEST FEATURES ==========")
        print(f"================={ev} = {selector[best_feature]} ================================")
        
    print(f"FINAL FEATURES")
    print(initial_feat)
    return total_log

log = select_k_best(k = 7)

with open('log.txt', mode='w') as f:
    f.write(str(log))

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [53]:
np.tile(np.array([3]), (len([4,5]),))


array([3, 3])

In [40]:
a = np.array([[],[],[],[]])
np.expand_dims(a, axis=1)
a

array([], shape=(4, 0), dtype=float64)